# K-Nearest Neighbor (KNN)

Ziel der vorliegenden Aufgabe ist es einen KNN-Kassifikator zu implementieren und auf den Titanic Datensatz anzuwenden. Dabei sollen Sie insbesondere untersuchen welcher Merkmalsraum und welches Ähnlichkeitsmaß für diese Aufgabe geeignet ist. 

Der KNN-Klassifikator ist eine einfache, parameterfreie Methode, bei welcher zu jedem Testvektor $\vec x_q$ die $k$-nächsten-Nachbarn, $\{\vec {x_t^1}...\vec {x_t^k}\}$ im Trainingsbestand, unter Berücksichtigung eines frei zu definierenden Ähnlichkeitsmaß, ermittelt werden. Die Klassenzugehörigkeit wird über einen einfachen Mehrheitsentscheid der nächsten $k$-Nachbarn für $\vec x_q$ prädiziert (siehe auch https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm).

### Imports

In [1]:
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
import pandas as pd
import itertools
%matplotlib inline

### Daten einlesen

In [2]:
DATA_FILE = './Data/original_titanic.csv'
df_original = pd.read_csv(DATA_FILE)
#identify columns with NaN values
print(df_original.isna().sum())

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
Home-Dest       564
dtype: int64


### Vorbereitungen

Führen Sie hier folgende Bearbeitungsschritte durch:
* (1) Datenlücken interpolieren,
* (2) Datensatz stochastisch verändern, 
* (3) Aufteilung in Trainings- und Testdatensatz.

Sie können Ihre Implementierungen der vorherigen Arbeitsblätter nutzen.

In [19]:
# (1) Datenlücken interpolieren
def prepareData(df):
    df.loc[np.isnan(df["Age"]) & (df['Sex']=='female'), 'Age'] = df.loc[df['Sex']=='female', 'Age'].mean()
    df.loc[np.isnan(df["Age"]) & (df['Sex']=='male'), 'Age'] = df.loc[df['Sex']=='male', 'Age'].mean()
    df.loc[:,'Fare'].interpolate(method='pad', inplace= True) #using existing values to fill NaN values
    return df
df_prepared = prepareData(df_original)
print(df_prepared.isna().sum())

# (2) Datensatz stochastisch verändern
df_shuffled =  df_prepared.sample(frac = 1).reset_index(drop=True) 

# (3) Aufteilung in Trainings- und Testdatensatz
len_all = len(df_shuffled)
len_train = round(len_all * 0.8, 0)
len_test = round(len_all * 0.2, 0)

df_train = df_shuffled[: int(len_train)]
df_test  = df_shuffled[: int(len_test)]

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
Home-Dest       564
dtype: int64


### Merkmale standardisieren
Die <i>Standardisierung</i> verleiht den Daten die Eigenschaften einer Standardnormalverteilung. Der Mittelwert jeder Merkmalsspalte beträgt 0, die Standardabweichung jeder Merkmalsspalte beträgt 1. Um zum Beispiel das Merkmal j zu standardiesieren, wird der Mittelwert $\mu$ der jeweiligen Stichprobe abgezogen und das Ergebnis durch die Standardabweichung $\sigma$ dividiert. Das Standardisierungsverfahren wird auf die Merkmalsspalten der Datenmenge einzeln angewendet. Siehe auch: https://en.wikipedia.org/wiki/Standard_score <br>

$x_j^{\prime(i)} = \frac{x_j^{(i)} - \mu_j}{\sigma_j}$. 

Implementieren sie die Funktion <b>normalize()</b> welche die Standardisierung (anhand Mittelwert und Standardabweichung) des Trainingsdatensatzes durchführt. Die Methoden-Parameter können Sie entsprechend Ihrer Implementierung erweitern. Überlegen Sie sich hierbei, welche Merkmale zur Standardisierung geeignet sind und welche nicht. Sie können Ihre Implementierung testen, indem Sie die Werte Mittelwert und Standardabweichung Ihrer standardisierten Merkmalsspalten prüfen. Der Mittelwert sollte 0 sein, die Standardabweichung 1. Geben Sie die Werte aus.

Speichern Sie sich hierbei die Werte für Mittelwert und Standardabweichung des Trainingsdatensatzes in einer geeigneten Datenstruktur. Führen sie die Standardisierung des Testdatensatzes anhand der Werte von Mittelwert und Standardabweichung vom Trainingsdatensatzes durch. <br>

<b>Wichtiger Hinweis</b>: Implementieren Sie die Funktion eigenständig, eine Standard-Funktion aus einem Framework ist nicht zulässig, wie bspw. *sklearn.preprocessing.StandardScaler*. Zum Testen Ihrer Implementierung können Sie diese Funktion nutzen. Standard-Numpy-Funktion zur Berechnung von *Mittelwert* und *Standardabweichung* sind selbstverständlich zugelassen. 

In [30]:
def normalize(df, cols):
    df_normalized = df.copy()
    
    for col_name in cols:
        
        #u = df_normalized.loc[:,col_name].mean()
        u = np.mean(df_normalized.loc[:,col_name])
        var = np.var(df_normalized.loc[:, col_name])
        o = np.sqrt(var)
        print("{}:\n---\nMittelwert: {}\nVarianz: {}\nStandartabweichung: {}".format(col_name, u, var, o))

        for idx, val in enumerate(df_normalized.loc[:,col_name]):           
            df_normalized.loc[idx, col_name] =  (val - u )/o

        u_strich = 1/o*(u-u) # siehe wikipedia
        var_strich = np.var(df_normalized[col_name])
        o_strich = np.sqrt(var_strich)
        print("Mittelwert' : {}\nVarianz' : {}\nStandartabweichung' : {}\n".format(u_strich, var_strich, o_strich))
    return df_normalized



#kategorisch-ordinale Merkmale: Age, SibSp, parch 
#quantitative merkmale: Fare
cols_to_normalize = ["Age", "SibSp", "Parch", "Fare"]
print("Train:")
df_train_norm = normalize(df_train, cols_to_normalize)  

print("\n-------------------------------\n")
print("Test:")
df_test_norm = normalize(df_test, cols_to_normalize)

Train:
Age:
---
Mittelwert: 29.945576613782617
Varianz: 162.3745813822387
Standartabweichung: 12.742628511505728
Mittelwert' : 0.0
Varianz' : 1.0
Standartabweichung' : 1.0

SibSp:
---
Mittelwert: 0.49570200573065903
Varianz: 1.13441323689187
Standartabweichung: 1.0650883704612824
Mittelwert' : 0.0
Varianz' : 1.0
Standartabweichung' : 1.0

Parch:
---
Mittelwert: 0.384909264565425
Varianz: 0.7907178284432986
Standartabweichung: 0.8892231600916042
Mittelwert' : 0.0
Varianz' : 0.9999999999999998
Standartabweichung' : 0.9999999999999999

Fare:
---
Mittelwert: 32.896748137535816
Varianz: 2569.379393477339
Standartabweichung: 50.689046089636946
Mittelwert' : 0.0
Varianz' : 1.0000000000000004
Standartabweichung' : 1.0000000000000002


-------------------------------

Test:
Age:
---
Mittelwert: 30.660582002464718
Varianz: 157.95146912836128
Standartabweichung: 12.567874487293437
Mittelwert' : 0.0
Varianz' : 1.0
Standartabweichung' : 1.0

SibSp:
---
Mittelwert: 0.583969465648855
Varianz: 1.50249

In [31]:
# from sklearn.preprocessing import StandardScaler
# def test_normalize(df, cols):
#     for col_name in cols:
#         scaler = StandardScaler()
#         scaler.fit(np.array(df[col_name]).reshape(-1, 1))
#         u = scaler.mean_
#         var = scaler.var_
#         o = np.sqrt(var)
#         print("{}:\n---\nMittelwert: {}\nVarianz: {}\nStandartabweichung: {}\n".format(col_name, u, var, o))

# test_normalize(df_train, cols_to_normalize)      

### Merkmalsraum definieren
Unabhängig von der eingesetzten Methodik zur Klassifikation ist es wichtig, dass Sie sich Gedanken über den Merkmalsraum machen. Konkret bedeutet das, dass sie definieren müssen wie ein Merkmalsvektor aussieht und mit welchem Ähnlichkeitsmaß Merkmalsvektoren verglichen werden. Sie sollten sich dabei mit den folgenden Fragen auseinander setzen:
* Welche einzelnen Attribute aus einem Objekt des Datensatzes sollen verwendet werden?
* Welche Probleme ergeben sich aufgrund von unterschiedlichen Skalenniveaus der Attribute? Wie können Sie diesen begegnen? 
* Wie gehen Sie mit kategoriellen Merkmalen um?
* Welches Ähnlichkeitsmaß setzen Sie ein? 

### Merkmalsraum definieren- Antwort
Welche einzelnen Attribute aus einem Objekt des Datensatzes sollen verwendet werden?
* Age, PClass, Sex, ggf. Fare

Welche Probleme ergeben sich aufgrund von unterschiedlichen Skalenniveaus der Attribute? Wie können Sie diesen begegnen? 
* Bspw. Fare und Age haben größere werte als Parch und SibSp. 
* TODO

Wie gehen Sie mit kategoriellen Merkmalen um?
* Darunter sind 2 kategorische Werte. Diese können auf numerische Werte übertragen werden. Bspw. Female = 0, Male = 1

Welches Ähnlichkeitsmaß setzen Sie ein? 
* Euklidische Distanz-> "Luftlinie"

### Merkmalsvektor extrahieren und normalisieren bzw. standardisieren
Schreiben sie eine Methode welche aus einer gegebenen Datenreihe einen Merkmalsvektor extrahiert. D.h. der Input ist eine Reihe aus dem Datensatz, der Rückgabewert ein Vektor bestehend aus den Daten.

In [98]:
def extractFeatureVector(row):
    # TODO : implement
    cl = float(row.loc['Pclass'])
    s = 0.0
    if row.loc['Sex'] == 'female': 
        s = 1.0
    x = np.array((float(row.loc['Age']),cl, s, row.loc['Fare'])).reshape(-1, 1)
    return x

Testen Sie die Funktion auf einem beliebigen Objekt des Datensatzes. Überprüfen Sie ob das Resultat Ihren Erwartungen entspricht. 

In [99]:
print(extractFeatureVector(df_train_norm.iloc[0]))


[[ 1.6522826]
 [ 3.       ]
 [ 0.       ]
 [-0.5098251]]


Wenn Sie einen nieder-dimensionalen Merkmalsraum, bspw. zwei-dimensional, gewählt haben, lässt sich dieser sehr komfortabel visualisieren. Zum Beispiel:

<img src="./Figures/titanic-nieder-dimensional.png" alt="drawing" style="width:400px;"/>

## Implementierung
Die Implementierung erfolgt innerhalb der Klasse <b>KNN</b>. Im folgenden werden die einzelnen Methoden und deren Funktionsweise kurz vorgestellt. <br>


### Konstruktor
Das KNN-Objekt wird mit dem Wert <b>k</b> initialisiert. Dieser bestimmt die Anzahl der zu betrachtenden Nachbarn. Wählen Sie k=3 als Wert.


### distance()-Methode:
In dieser Methode gilt es eine Funktion zu implementieren, welche die Ähnlichkeit zweier Merkmalsvektoren vergleicht. Diese Methode soll die Ähnlichkeit der zwei Merkmalsvektoren, welche als Methoden-Parameter übergeben werden, bestimmen. Wählen Sie hierbei die aus der Vorlesung bekannten Distanz-Funktionen wie bspw. *Euklidische Distanz*, *Manhatten Distanz*, *Minkovski Distanz* etc. <br>

<b>Auch hier gilt</b>: Implementieren Sie die Funktion eigenständig, eine Standard-Funktion aus einem Framework ist nicht zulässig, wie bspw. *sklearn.metrics.pairwise.euclidean_distances*. Zum Testen Ihrer Implementierung können Sie diese Funktionen nutzen.


### fit()-Methode:
Als Methoden-Parameter dient der normierte Trainingsdatensatz. In dieser Methode soll das Modell anhand der Trainingsdaten gebildet werden mit dem entsprechenden zuvor definierten Merkmalsvektor. <br>
Stellen Sie sicher, dass in der Liste <b>self.trainData</b> die Merkmalsvektoren aus dem Trainingsdatensatz enthalten sind. <br>
Stellen sie sicher, dass in der Liste <b>self.trainLabel</b> die Zielwerte des Merkmals *survived* aus dem Trainingsdatensatz enthalten sind.


### predict()-Methode:
Als Methoden-Parameter dient ein Merkmalsvektor. Implementieren Sie den in der Vorlesung besprochenen Algorithmus für den KNN-Klassifikator. Der Rückgabewert der Methode ist die entsprechende Mehrheits-Entscheidung.

In [95]:
class KNN(object):
    trainLabel = None
    trainLabel = None
    k = 1
    distances = []
    
    def __init__(self, k):
        self.k = k

    #annahme das v1 und v2 dieselbe form haben
    # sqrt(dot(x, x) - 2 * dot(x, y) + dot(y, y))
    def distance(self, vector1, vector2 ):

#         i1 = np.array(vector1)
#         i2 = np.array(vector2)
#         return np.linalg.norm(i1 - i2)
#         return ( np.sqrt(vector1.dot(vector1) - (2* vector1.dot(vector2)) + vector2.dot(vector2)) )

#         manhattan distance 
#         return sum(abs(e1-e2) for e1, e2 in zip(vector1, vector2))
    

#        minkowski distance
         p = 1
         return sum(abs(e1-e2)**p for e1, e2 in zip(vector1, vector2))**(1/p)
    
    
    def fit(self, data, label):
        self.trainData = data
        self.trainLabel = label
       
        

    def predict(self, x):
        #todo das funktioniert nicht, weil dabei die labels nicht ebrücksichtig werden
        for idx, trainSample in enumerate(self.trainData):
            dist = self.distance(trainSample, x)
            self.distances.append((trainSample, dist, self.trainLabel[idx]))
        self.distances.sort(key= lambda x: x[1]) #sort for minimal distance
        return self.distances[:self.k]

## Training und Test des Algorithmus

Führen die Modellbildung ("Training") anhand der KNN-Klasse und der <b>fit()</b>-Methode durch. <br>
Die <b>predict()</b>-Methode soll einen Merkmalsvektor $\vec x_q$  auf die entsprechende Klassenzugehörigkeit $l \in \{0,1\}$ abbilden. <br>
Testen Sie die <b>predict()</b>-Methode mit den von ihnen gewählten Merkmalsvektoren ( _Hinweis_ : Einsatz der <b>extractFeatureVector()</b>-Methode) aus dem normierten Testdatensatz. Ermitteln sie hierzu die Korreklassifizierungsrate.

In [96]:
#init
data = []
labels = []
l_train = len(df_train_norm)
i = 0
while i < l_train:
    data.append(extractFeatureVector(df_train_norm.iloc[i]))
    labels.append(df_train_norm.loc[i, 'Survived'])
    i += 1
train_data = np.array(data)
train_label = np.array(labels)

knn = KNN(3)
knn.fit(train_data, train_label)

In [97]:
##test
l_test = len(df_test_norm)
i = 0
tp = 0
fp = 0
fn = 0
tn = 0

while i < l_test:
    row = df_test_norm.iloc[i]
    label = int(df_test_norm.loc[i, 'Survived'])
    sortedNeighbors = knn.predict(extractFeatureVector(row))
  

    for neighbor in sortedNeighbors:
        if label ==  neighbor[2]:
            if label == 1:
                tp +=1
            else:
                tn += 1

        else: 
            if label == 1:
                fn +=1
            else:
                fp += 1
    i+=1
#get acc           
print("True Positives: %d" %tp)   
print("True Negatives: %d" %tn)

print("False Positives: %d" %fp)
print("False Negatives: %d" %fn)

acc = (tp + tn) / (tp + tn + fp + fn)
print("Accurency: %f" %acc)            

True Positives: 0
True Negatives: 486
False Positives: 0
False Negatives: 300
Accurency: 0.618321


## Weitere Evaluation
Untersuchen Sie wie sich die Klassifikationsleistung in Abhängigkeit von verschiedenen Änhlichkeitsmaßen bzw. Merkmalsvektoren verhält. 

## Euclidian Distance
### Nur Geschlecht
True Positives: 0
True Negatives: 486
False Positives: 0
False Negatives: 300
Accurency: 0.618321
    
### Pclass, Sex
True Positives: 0
True Negatives: 486
False Positives: 0
False Negatives: 300
Accurency: 0.618321

### Age, Sex
True Positives: 0
True Negatives: 486
False Positives: 0
False Negatives: 300
Accurency: 0.618321


### Pclass, Sex, Age
True Positives: 95
True Negatives: 334
False Positives: 152
False Negatives: 205
Accurency: 0.545802


### Age, Sex, Pclass, Fare
True Positives: 3
True Negatives: 483
False Positives: 3
False Negatives: 297
Accurency: 0.618321

Mit Minkowski Distance p = 2
True Positives: 3
True Negatives: 483
False Positives: 3
False Negatives: 297
Accurency: 0.618321

## City Block Distance

### Nur Geschlecht
True Positives: 0
True Negatives: 486
False Positives: 0
False Negatives: 300
Accurency: 0.618321


### Pclass, Sex, Age
True Positives: 95
True Negatives: 335
False Positives: 151
False Negatives: 205
Accurency: 0.547074

### Age, Sex, Pclass, Fare
True Positives: 1
True Negatives: 485
False Positives: 1
False Negatives: 299
Accurency: 0.618321

## Fazit
Klassifikationsleitung bleibt gleich unabhänig des Distanzmaß